# PyTorch COCO Tools Profiler Utility


In [ ]:
import logging
from pathlib import Path
import sys

import torch
import torch.nn as nn


# Add the src directory to the path to import the utility
# Adjust the path if necessary based on where you run the notebook
module_path = Path().resolve().parent / "src"
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))


logging.basicConfig(level=logging.INFO)

## Setup Model and Inputs

In [ ]:
from torch import Tensor


def my_complex_function(x: Tensor, y: Tensor) -> Tensor:
    # Some operations
    result = x * y + torch.sin(x)
    result = result / (torch.cos(y) + 1e-6)
    # More operations
    for _ in range(3):
        result = torch.relu(result)
    return result

# Profile

In [ ]:
from pytorchcocotools.utils.profiler import (
    CombinedProfiler,
    ProfilerActivity,  # pyright: ignore[reportPrivateImportUsage]
)

profiler = CombinedProfiler(
    functions_to_profile=[my_complex_function],
    activities=[
        ProfilerActivity.CPU,
        ProfilerActivity.CUDA,
    ],
)
with profiler.profile() as prof:
    a = torch.randn(64, 100)
    b = torch.randn(64, 100)
    model = nn.Linear(100, 10)

    if torch.cuda.is_available():
        a = a.cuda()
        b = b.cuda()
        model = model.cuda()

    result = my_complex_function(a, b)